## Nonpolar solvation free energy with an implicit solvent model

This notebook is an implementation of the implicit solvent model described in "A Simple Electrostatic Model for the Hard-Sphere Solute Component of Nonpolar Solvation" (Christopher D. Cooper and Jaydeep P. Bardhan) for the nonpolar component of solvation free energy ($\Delta G_{np}$). This model decomposes $\Delta G_{np}$ in the free energy spent in the cavity creation ($\Delta G_{cav}$), plus the energy of the solute-solvent dispersion interaction ($\Delta G_{disp}$), so that $\Delta G_{np} = \Delta G_{cav} + \Delta G_{disp}$.

### A capacitor model for $\Delta G_{cav}$

The cavity energy model is based in a simple *capacitor* model, that models the solute as two concentric conducting surfaces, one on the dielectric interface (the solvent-excluded surface) and another one around the first solvation shell, one water radius out (on the solvent-accessible surface). Then, $\Delta G_{cav}$ is the electrostatic energy stored in the capacitor, such that it sustains a potential $\phi_{static}$ inside the dielectric interface and 0 outside the shell.

The electrostatic potential due to a surface charge $\sigma$ is

\begin{equation}
\phi(\mathbf{r}) = \oint_\Gamma \frac{\sigma(\mathbf{r}')}{4\pi\epsilon|\mathbf{r}-\mathbf{r}'|}d\mathbf{r}' = \frac{1}{\epsilon}V_\mathbf{r}(\sigma)
\end{equation}

Then, the surface charge densities on the SES and SAS ($\sigma_{SES}$ and $\sigma_{SAS}$) required to have potential $\phi_{static}$ on the SES and 0 on the SAS is solution to

\begin{equation}
\left[
\begin{matrix}
V_{diel} & V_{diel} \\
V_{shell} & V_{shell} 
\end{matrix}
\right]
\left(
\begin{matrix}
\sigma_{diel}\\
\sigma_{shell}
\end{matrix}
\right)
=
\epsilon_{shell}
\left(
\begin{matrix}
\phi_{static}\\
0
\end{matrix}
\right)
\end{equation}

where $\epsilon_{shell}$ is the dielectric constant in the first hydration shell. Having the surface charges, the energy stored in the capacitor is

\begin{equation}
\Delta G_{cav} = \oint \phi_{static} \sigma_{shell}(\mathbf{r}) d\mathbf{r}
\end{equation}

Next, we will use the [`bempp`](https://bempp.com/) library to solve the matrix equation above numerically, and compute $\Delta G_{cav}$.

First, we import the required libraries and define a function to generate the grid in `bempp` format. This reads meshes in [`msms`](https://mgl.scripps.edu/people/sanner/html/msms_home.html) format (`.vert` and `.face` files).

In [1]:
import numpy
import bempp.api
from bempp.api.operators.boundary import sparse, laplace
from scipy.sparse.linalg import gmres

def generate_grid(filename):
    
    face = open(filename+'.face','r').read()
    vert = open(filename+'.vert','r').read()

    faces = numpy.vstack(numpy.char.split(face.split('\n')[0:-1]))[:,:3].astype(int) - 1
    verts = numpy.vstack(numpy.char.split(vert.split('\n')[0:-1]))[:,:3].astype(float)

    grid = bempp.api.grid_from_element_data(verts.transpose(), faces.transpose())
    
    N = grid.leaf_view.entity_count(0)
    elements = list(grid.leaf_view.entity_iterator(0))
    area = numpy.zeros(N)

    # remove zero areas
    for i in range(N):
        area[i] = elements[i].geometry.volume

    area_nonzero = numpy.where(area>1e-12)[0]
    
    faces_nonzero = faces[area_nonzero,:]
    
    grid = bempp.api.grid_from_element_data(verts.transpose(), faces_nonzero.transpose())
    
    return grid

Could not find Gmsh.Interactive plotting and shapes module not available.


Next, we define a function to generate the right-hand side of the matrix equation, and another that will assemble the matrix and solve the linear system

In [3]:
def rhs_fun(x, n, domain_index,result):
    global phi_static
    result[:] = phi_static

def solve_matrix(mesh_diel, mesh_stern, phi_static, eps_s):

    grid_diel = generate_grid(mesh_diel)
    grid_stern = generate_grid(mesh_stern)

    space_diel = bempp.api.function_space(grid_diel, "DP", 0)
    space_stern = bempp.api.function_space(grid_stern, "DP", 0)
    
    N_diel = grid_diel.leaf_view.entity_count(0)
    N_stern = grid_stern.leaf_view.entity_count(0)
    
    elements_d = list(grid_diel.leaf_view.entity_iterator(0))
    elements_s = list(grid_stern.leaf_view.entity_iterator(0))

    phis_grid_fun = bempp.api.GridFunction(space_diel, fun=rhs_fun)

    rhs = numpy.concatenate([eps_s*phis_grid_fun.coefficients, 
                      numpy.zeros(N_stern)])
    

    M11   = laplace.single_layer(space_diel, space_diel, space_diel) 
    M12   = laplace.single_layer(space_stern, space_diel, space_diel) 
    M21   = laplace.single_layer(space_diel, space_stern, space_stern)
    M22   = laplace.single_layer(space_stern, space_stern, space_stern) 

    blocked = bempp.api.BlockedOperator(2, 2)
    blocked[0,0] = M11
    blocked[0,1] = M12
    blocked[1,0] = M21
    blocked[1,1] = M22
    op_discrete = blocked.strong_form()
    
    sigma, info = gmres(op_discrete, rhs, tol=1e-5, maxiter=500, restart = 1000)
    
    if info>0:
        print ('Not converged, %i iterations'%info)
    elif info<0:
        print ('Solver diverges')
    
    sigma_d = sigma[:N_diel]
    sigma_s = sigma[N_diel:]
    
    return sigma_d, elements_d

Finally, we can compute the energy

In [4]:
def compute_energy_cav(mesh_diel, sigma_diel, elements_diel, phi_static):
    
    qe = 1.60217662e-19
    Rw = 1.4
    Na = 6.0221409e23
    eps_0 = 8.854187817e-12
    
    N_diel = len(sigma_diel)
    area_d = numpy.zeros(N_diel)
    for i in range(N_diel):
        area_d[i] = elements_diel[i].geometry.volume
    
    energy = 0.5*numpy.sum(area_d*sigma_d*phi_static)

    return energy

Now, we can run as follows, where `mesh_ses` and `mesh_sas` are the `msms` mesh files of the SES and SAS, respectively

In [5]:
mesh_shell = 'mobley_test/nitroethane/surf_d02_stern'
mesh_diel = 'mobley_test/nitroethane/surf_d02'

phi_static = 10.7 #kcal/mol/e
phi_static *= 4.184 # kJ/mol/e

eps_s = 7.38

sigma_diel, elements_diel = solve_matrix(mesh_ses, mesh_diel, phi_static, eps_s)
dGcav = compute_energy_cav(mesh_diel, sigma_diel, elements_diel, phi_static)

NameError: name 'mesh_ses' is not defined

### A continuum integral model for $\Delta G_{disp}$

The solute-solvent dispersion interaction can be computed by integrating the Lennard-Jones potential in the solvent

\begin{equation}
\Delta G_{disp} = \sum_i \int_{solv}\rho_w \left(\frac{A_i}{|\mathbf{r}-\mathbf{r}_i|^12} - \frac{B_i}{|\mathbf{r}-\mathbf{r}_i|^6}\right) d\mathbf{r}
\end{equation}

where the sum is over the solute molecules, $A_i$ and $B_i$ are the Lennard-Jones parameters for atom $i$, and $\rho_w = 0.0336 angs^{-3}$ is the solvent number density. Using the divergence theorem, we write this equation as

\begin{equation}
\Delta G_{disp} = \sum_i \oint_{shell} \rho_w \frac{\partial}{\partial \mathbf{n}}\left( \frac{A_i}{90|\mathbf{r} - \mathbf{r}_i|^{10}} - \frac{B_i}{12|\mathbf{r} - \mathbf{r}_i|^4}\right) \text{d}\mathbf{r}  
\end{equation}

To account for the fact that the water number density increases near surfaces, we add an extra  1.4$angs$ layer beyond the shell surface, and with an augmented density of 1.8$\rho_w$. Then, the energy can be written as

\begin{equation}
\Delta G_{disp} = \sum_i \int_{layer}1.8\rho_w \left(\frac{A_i}{|\mathbf{r}-\mathbf{r}_i|^12} - \frac{B_i}{|\mathbf{r}-\mathbf{r}_i|^6}\right) d\mathbf{r} + \int_{solv}\rho_w \left(\frac{A_i}{|\mathbf{r}-\mathbf{r}_i|^12} - \frac{B_i}{|\mathbf{r}-\mathbf{r}_i|^6}\right) d\mathbf{r}  
\end{equation}

where the first term is the integral over the new layer, and the second over the rest of the solvent. We can write the first integral as

\begin{equation}
\int_{layer}1.8\rho_w \left(\frac{A_i}{|\mathbf{r}-\mathbf{r}_i|^12} - \frac{B_i}{|\mathbf{r}-\mathbf{r}_i|^6}\right) d\mathbf{r} = \int_{layer+solv}1.8\rho_w \left(\frac{A_i}{|\mathbf{r}-\mathbf{r}_i|^12} - \frac{B_i}{|\mathbf{r}-\mathbf{r}_i|^6}\right) d\mathbf{r} - \int_{solv}1.8\rho_w \left(\frac{A_i}{|\mathbf{r}-\mathbf{r}_i|^12} - \frac{B_i}{|\mathbf{r}-\mathbf{r}_i|^6}\right) d\mathbf{r}
\end{equation}

which can be written as surface integrals

\begin{equation}
\int_{layer}1.8\rho_w \left(\frac{A_i}{|\mathbf{r}-\mathbf{r}_i|^12} - \frac{B_i}{|\mathbf{r}-\mathbf{r}_i|^6}\right) d\mathbf{r} = \oint_{shell} 1.8\rho_w \frac{\partial}{\partial \mathbf{n}}\left( \frac{A_i}{90|\mathbf{r} - \mathbf{r}_i|^{10}} - \frac{B_i}{12|\mathbf{r} - \mathbf{r}_i|^4}\right) \text{d}\mathbf{r} - \oint_{shell, out} 1.8\rho_w \frac{\partial}{\partial \mathbf{n}}\left( \frac{A_i}{90|\mathbf{r} - \mathbf{r}_i|^{10}} - \frac{B_i}{12|\mathbf{r} - \mathbf{r}_i|^4}\right) \text{d}\mathbf{r}. 
\end{equation}

Then, the dispersion energy is

\begin{equation}
\Delta G_{disp} = \sum_i \left( \oint_{shell} 1.8\rho_w \frac{\partial}{\partial \mathbf{n}}\left( \frac{A_i}{90|\mathbf{r} - \mathbf{r}_i|^{10}} - \frac{B_i}{12|\mathbf{r} - \mathbf{r}_i|^4}\right) \text{d}\mathbf{r} -  \oint_{shell, out} 0.8\rho_w \frac{\partial}{\partial \mathbf{n}}\left( \frac{A_i}{90|\mathbf{r} - \mathbf{r}_i|^{10}} - \frac{B_i}{12|\mathbf{r} - \mathbf{r}_i|^4}\right) \text{d}\mathbf{r} \right)
\end{equation}

First, we need to import the mesh for the outer interface, between the new layer and the solvent, and the position and van der Waals parameters of the atoms

In [ ]:
mesh_shell_out = 'mobley_test/nitroethane/surf_d02_stern28'
atom_xyzr = 'mobley_test/nitroethane/nitroethane.xyzr'
atom_vdw = 'mobley_test/nitroethane/nitroethane.vdw'

and the following function computes the surface integral above

In [ ]:
def disp_energy(atom_xyzr_file, atom_vdw_file, mesh):

    qe = 1.60217662e-19
    Na = 6.0221409e23
    rho_w = 0.0336# 1/angs3 number density of water at standard conditions
    water_r = 1.7683 # angs
    water_eps = 0.1520 # kcal/mol
    
    atom_xyzr = numpy.loadtxt(atom_xyzr_file) 
    
    atom_pos = atom_xyzr[:,:3]
    atom_r   = atom_xyzr[:,3]
    N_atom = len(atom_r)
    atom_eps = numpy.zeros(N_atom)

    i=0
    for line in file(atom_vdw_file):
        line = line.split()
        atom_eps[i] = float(line[-1])
        i += 1

    grid = generate_grid(mesh)

    N_panel = grid.leaf_view.entity_count(0)
    vertices = grid.leaf_view.vertices 
    triangles = grid.leaf_view.elements
    elements = list(grid.leaf_view.entity_iterator(0))

    area = numpy.zeros(N_panel)
    center = numpy.zeros((N_panel,3))
    normal = numpy.zeros((N_panel,3))
    for i in range(N_panel):
        area[i] = elements[i].geometry.volume
        center[i,:] = numpy.average(elements[i].geometry.corners[:],axis=1)
        v1 = elements[i].geometry.corners[:,1] - elements[i].geometry.corners[:,0]
        v2 = elements[i].geometry.corners[:,2] - elements[i].geometry.corners[:,0]
        normal[i,:] = numpy.cross(v1,v2)/(2*area[i]) 

    integral_i = numpy.zeros(N_atom)
    for i in range(N_atom):
        r_local = center - atom_pos[i,:]
        r_norm = numpy.sqrt(numpy.sum(r_local**2, axis=1))
        rdn = numpy.sum(r_local*normal, axis=1)
        epsilon = numpy.sqrt(water_eps*atom_eps[i])
    
        A = epsilon*(water_r + atom_r[i])**12
        B = 2*epsilon*(water_r + atom_r[i])**6
        
        r_large = numpy.where(r_norm>0.4*atom_r[i])[0]
        if len(r_large)!=len(r_norm):
            print 'Triangles out: %i'%(len(center)-len(r_large))
    
        integral_i[i] = numpy.sum((A/(9*r_norm[r_large]**12) - B/(3*r_norm[r_large]**6)) * rdn[r_large] * area[r_large])   

    energy = rho_w*numpy.sum(integral_i) #kcal/mol
    return energy

Next, we call this function for both surfaces, and compute the energy

In [ ]:
e_diel = vdw_energy_mol(atom_xyzr, atom_vdw, mesh_stern)
e_diel_out = vdw_energy_mol(atom_xyzr, atom_vdw, mesh_stern_out)

and finally compute the energy

In [ ]:
dGdisp = e_diel*1.8 - e_diel_out*0.8